In [ ]:
pip install layoutparser
# install detectron2
!pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2' 
 # install OCR module
 !pip install layoutparser[ocr]  

In [1]:
 import layoutparser as lp
 import matplotlib.pyplot as plt
 import matplotlib
 %matplotlib inline
 import cv2 

In [6]:
model = lp.Detectron2LayoutModel('lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config', 
                                  extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
                                  label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})

config.yaml?dl=1: 8.19kB [00:01, 6.74kB/s]
model_final.pth?dl=1: 856MB [00:14, 59.4MB/s]                           


In [7]:
image = cv2.imread("Page_12.jpg")
image = image[..., ::-1] 
    # Convert the image from BGR (cv2 default loading style)
    # to RGB

In [8]:
layout = model.detect(image)
    # Detect the layout of the input image

/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:101: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


In [ ]:
lp.draw_box(image, layout, box_width=10)
    # Show the detected layout of the input image

In [15]:
layout[1]

TextBlock(block=Rectangle(x_1=2798.92529296875, y_1=519.5648803710938, x_2=3476.47705078125, y_2=575.7400512695312), text=None, id=None, type=Title, parent=None, next=None, score=0.9951589703559875)

In [10]:
text_blocks = lp.Layout([b for b in layout if b.type=='Text'])
figure_blocks = lp.Layout([b for b in layout if b.type=='Figure'])

In [11]:
text_blocks = lp.Layout([b for b in text_blocks \
                   if not any(b.is_in(b_fig) for b_fig in figure_blocks)])

In [12]:
h, w = image.shape[:2]

left_interval = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(image)

left_blocks = text_blocks.filter_by(left_interval, center=True)
left_blocks.sort(key = lambda b:b.coordinates[1], inplace=True)
# The b.coordinates[1] corresponds to the y coordinate of the region
# sort based on that can simulate the top-to-bottom reading order 
right_blocks = lp.Layout([b for b in text_blocks if b not in left_blocks])
right_blocks.sort(key = lambda b:b.coordinates[1], inplace=True)

# And finally combine the two lists and add the index
text_blocks = lp.Layout([b.set(id = idx) for idx, b in enumerate(left_blocks + right_blocks)])

In [ ]:
lp.draw_box(image, text_blocks,
            box_width=14, 
            show_element_id=True)


In [14]:
ocr_agent = lp.TesseractAgent(languages='eng') 
    # Initialize the tesseract ocr engine. You might need 
    # to install the OCR components in layoutparser:
    # pip install layoutparser[ocr]

In [ ]:
!sudo apt install tesseract-ocr

In [15]:
for block in text_blocks:
    segment_image = (block
                       .pad(left=5, right=5, top=5, bottom=5)
                       .crop_image(image))
        # add padding in each image segment can help
        # improve robustness 
        
    text = ocr_agent.detect(segment_image)
    block.set(text=text, inplace=True)

In [16]:
for txt in text_blocks.get_texts():
    print(txt, end='\n---\n')

Partnerships and collaboration are crucial to achieving our
sustainability ambitions. By pooling knowledge and expertise, from
both within our industry and outside the sector, we continue to
create solutions that add value for the client as well as address
environmental and societal challenges. By participating in research
or collaboration projects with other players in the maritime sector,
we increase the likelihood of new sustainable technologies,
concepts or approaches being approved by legislators or brought
to scale, enabling us to offer these solutions to our clients.

Entering into dialog with both our external and internal stakeholder
enables us to acquire a clear understanding of their expectations
and interests and allows us to establish and better understand
material topics. While we have a wide range of stakeholders, we
see our key stakeholder groups as being: employees; clients and
their respective project communities; suppliers and subcontractors;
NGOs and civil society o